只取7-8/16-17的每个link的小时average travel time作为feature
所以 对于具体的link，只要 7-8/16-17里的任意一个时间窗口有数据，这个时间窗口的average travel time就能作为小时的average travel time
如果小时的三个时间窗口都没有数据，那么这一行直接去掉

对于label的处理同样的，只要小时的任意一个时间窗口有数据，这个时间窗口的average travel time就能作为小时的average travel time
如果小时的三个时间窗口都没有数据，那么这一行直接去掉

暂时没有用小时的车流量，这里有个问题就是，像travel time的话，只要这个小时里的一个时间窗口有数值，我就把这个时间窗口的average travel time作为小时的average travel time。但像车流量是累加的，所以时间窗口内数据的缺失会带来问题。我观察过数据，时间窗口内的数据缺失大致有两个原因，一个原因是这个时间窗口内这个link，它就是没有车通过，那么这种情况下，该时间窗口的车流量可以直接填为0，这没问题；但另一种情况是，数据本身的缺失，就比如说我忘记了哪条路的哪天了，table5里下午4:00-4:14的数据都是有的，然后突然4:14之后的数据就没了，一直到之后一两个小时才又有数据。但要把这两种情况区分开，其实挺麻烦的。


中秋跟国庆的数据已经去掉，然后特殊日的one-hot还没加，降雨跟星期几的信息已加

最后的数据量，以及分布，其实还好

时间窗口的平移，还没有做

A_2
length:323


weekday distribution （[数量，比例]
[47, 0.14551083591331268]
[48, 0.14860681114551083]
[48, 0.14860681114551083]
[44, 0.13622291021671826]
[44, 0.13622291021671826]
[44, 0.13622291021671826]
[48, 0.14860681114551083]


hour distribution
[81, 0.25077399380804954]
[81, 0.25077399380804954]
[80, 0.2476780185758514]
[81, 0.25077399380804954]


A_3
length:319


weekday distribution
[47, 0.14733542319749215]
[48, 0.15047021943573669]
[48, 0.15047021943573669]
[43, 0.13479623824451412]
[44, 0.13793103448275862]
[44, 0.13793103448275862]
[45, 0.14106583072100312]


hour distribution
[81, 0.25391849529780564]
[80, 0.2507836990595611]
[79, 0.2476489028213166]
[79, 0.2476489028213166]


B_1
length:265


weekday distribution
[39, 0.1471698113207547]
[37, 0.13962264150943396]
[38, 0.14339622641509434]
[39, 0.1471698113207547]
[34, 0.12830188679245283]
[35, 0.1320754716981132]
[43, 0.16226415094339622]


hour distribution
[70, 0.2641509433962264]
[74, 0.2792452830188679]
[62, 0.2339622641509434]
[59, 0.22264150943396227]


B_3
length:321


weekday distribution
[47, 0.14641744548286603]
[48, 0.14953271028037382]
[48, 0.14953271028037382]
[43, 0.13395638629283488]
[44, 0.13707165109034267]
[44, 0.13707165109034267]
[47, 0.14641744548286603]


hour distribution
[81, 0.2523364485981308]
[80, 0.24922118380062305]
[80, 0.24922118380062305]
[80, 0.24922118380062305]


C_1
length:275


weekday distribution
[38, 0.13818181818181818]
[38, 0.13818181818181818]
[38, 0.13818181818181818]
[35, 0.12727272727272726]
[42, 0.15272727272727274]
[39, 0.14181818181818182]
[45, 0.16363636363636364]


hour distribution
[74, 0.2690909090909091]
[71, 0.2581818181818182]
[70, 0.2545454545454545]
[60, 0.21818181818181817]


C_3
length:256


weekday distribution
[34, 0.1328125]
[36, 0.140625]
[41, 0.16015625]
[35, 0.13671875]
[31, 0.12109375]
[35, 0.13671875]
[44, 0.171875]


hour distribution
[59, 0.23046875]
[63, 0.24609375]
[65, 0.25390625]
[69, 0.26953125]

In [87]:
import math
from datetime import datetime,timedelta
import numpy as np
import pandas as pd
names=locals()

In [88]:
link_time={}
time_train={}
time_predict={}
time_check={}
test_time_train={}
test_time_check={}
weathers={}
rainingTotalTime={}

In [89]:
#构造各个link的travel time的dictionary
#有三层，第一层是link(100,101,102...)，第二层是日期[time.month*31+time.day] 月乘31+day的好处是，可以用一个数字来表达这个日期，方便后续操作
#而且从这个数字又可以返回日期 float(str(l//31)+'.'+str(l%31)，第三层是hour，最后存的是，不同link，在不同日期里的不同小时的travel time（存的是具体的数据，后续取平均就能得到小时的平均）
file_path='/Users/vayne/Desktop/dm_pro_engin/phase1_training/table5.csv'

# Step 1: Load trajectories
fr = open(file_path, 'r')
fr.readline()  # skip the header
traj_data = fr.readlines()
fr.close()

# 建立各Link時間的dictionary
for i in range(24):
    link_time[str(i+100)]={}
# Step 2: Create a dictionary to store travel time for each route per time window
travel_times = {}  # key: route_id. Value is also a dictionary of which key is the start time for the time window and value is a list of travel times
for i in range(len(traj_data)):
    each_traj = traj_data[i].replace('"', '').split(',')
    intersection_id = each_traj[0]
    tollgate_id = each_traj[1]

    route_id = intersection_id + '_' + tollgate_id
    if route_id not in travel_times.keys():
        travel_times[route_id] = {}

    trace_start_time = each_traj[3]
    travel_seq = each_traj[4]
    trace_start_time = datetime.strptime(trace_start_time, "%Y-%m-%d %H:%M:%S")
    time_window_minute = math.floor(trace_start_time.minute / 20) * 20
    start_time_window = datetime(trace_start_time.year, trace_start_time.month, trace_start_time.day,
                                     trace_start_time.hour, time_window_minute, 0)
    time=start_time_window
    tt = float(each_traj[-1])
    # 國慶不管，因道路狀態不一樣
    #if(time.month==10 and time.day in [1,2,3,4,5,6,7]):
     #   continue
    # 中秋節不管，因道路狀態不一樣
    if(time.month==9 and time.day in [15,16,17]):
        continue
    if start_time_window not in travel_times[route_id].keys():
        travel_times[route_id][start_time_window] = [tt]
    else:
        travel_times[route_id][start_time_window].append(tt)
    if(time.hour in [7,16]):
        try:
            #依序解析該筆紀錄於各link所花費的時間
            for each_seq in travel_seq.split(';'):
                each_seq=each_seq.split('#')
                link_id=each_seq[0]
                start_datetime=each_seq[1]
                start_datetime=datetime.strptime(start_datetime, "%Y-%m-%d %H:%M:%S")
                use_seconds=float(each_seq[2])
                if(time.month*31+time.day not in link_time[link_id]):
                    link_time[link_id][time.month*31+time.day]={}
                if(time.hour not in link_time[link_id][time.month*31+time.day]):
                    link_time[link_id][time.month*31+time.day][time.hour]=[]
                    
                link_time[link_id][time.month*31+time.day][time.hour].append(use_seconds)
        except:pass
                

In [90]:
#某个循环嵌套错了！！！
link_time['101'][236][7]

[6.54, 9.4]

In [91]:
date_key=link_time['100'].keys()

In [92]:
date_key

dict_keys([236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327])

In [93]:

#算label的小时平均时间
names=locals()
for route in travel_times.keys():
    names[str(route+'_'+'8_9')]=[]
    names[str(route+'_'+'9_10')]=[]
    names[str(route+'_'+'17_18')]=[]
    names[str(route+'_'+'18_19')]=[]
    
    

    
for route in travel_times.keys():
        route_time_windows = list(travel_times[route].keys())
        route_time_windows.sort()
        for time_window_start in route_time_windows:
            time_window_end = time_window_start + timedelta(minutes=20)
            tt_set = travel_times[route][time_window_start]
            avg_tt = round(sum(tt_set) / float(len(tt_set)), 2)
            
            if time_window_start.hour==8:
                outline = [route,time_window_start.month*31+time_window_start.day,time_window_start,str(avg_tt)]
                names[str(route+'_'+'8_9')].append(outline)
            
            if time_window_start.hour==9:
                outline = [route,time_window_start.month*31+time_window_start.day,time_window_start,str(avg_tt)]
                names[str(route+'_'+'9_10')].append(outline)
            
            if time_window_start.hour==17:
                outline = [route,time_window_start.month*31+time_window_start.day,time_window_start,str(avg_tt)]
                names[str(route+'_'+'17_18')].append(outline)
            
            if time_window_start.hour==18:
                outline = [route,time_window_start.month*31+time_window_start.day,time_window_start,str(avg_tt)]
                names[str(route+'_'+'18_19')].append(outline)
                

In [94]:
#构造label的list
route_dict=['A_2','A_3','B_1','B_3','C_1','C_3']
time_dict=['8_9','9_10','17_18','18_19']

for i in route_dict:
    for j in time_dict:
        print([i,j])
        print('missing date')
        names[str(i)+'_'+str(j)+'_'+'date']=[]
        for k in names[str(i)+'_'+str(j)]:
            names[str(i)+'_'+str(j)+'_'+'date'].append(k[1])
        names[str(i)+'_'+str(j)+'_'+'date']=list(dict.fromkeys(names[str(i)+'_'+str(j)+'_'+'date']))
        
        kk=0
        for l in date_key:
            if l not in names[str(i)+'_'+str(j)+'_'+'date']:
                print(float(str(l//31)+'.'+str(l%31)))
                kk+=1
        print('len:'+str(kk))


        names[str(i)+'_'+str(j)+'_'+'hr_avg']=[]
        for k in names[str(i)+'_'+str(j)+'_'+'date']:
            temp=[]
            temp1=0
            for m in names[str(i)+'_'+str(j)]:
                if m[1]==k:
                    temp.append(m)
                    temp1+=float(m[3])
            names[str(i)+'_'+str(j)+'_'+'hr_avg'].append([temp1/len(temp),i,temp[0][1],str(temp[0][2].month)+'-'+str(temp[0][2].day),temp[0][2].hour,temp[0][2].weekday()])
        print('\n')

['A_2', '8_9']
missing date
len:0


['A_2', '9_10']
missing date
len:0


['A_2', '17_18']
missing date
8.8
len:1


['A_2', '18_19']
missing date
len:0


['A_3', '8_9']
missing date
len:0


['A_3', '9_10']
missing date
7.21
len:1


['A_3', '17_18']
missing date
8.8
len:1


['A_3', '18_19']
missing date
9.4
len:1


['B_1', '8_9']
missing date
7.22
7.26
8.1
8.16
8.19
8.22
8.23
8.26
8.29
8.3
9.0
len:11


['B_1', '9_10']
missing date
8.1
8.13
8.2
8.24
8.26
8.28
8.3
len:7


['B_1', '17_18']
missing date
7.19
7.2
7.3
8.0
8.8
8.13
8.14
8.15
8.2
8.22
8.23
8.25
8.26
8.27
8.28
8.29
9.2
9.3
9.4
len:19


['B_1', '18_19']
missing date
7.2
7.21
7.26
7.27
7.28
7.3
8.2
8.3
8.5
8.9
8.1
8.12
8.13
8.18
8.19
8.22
8.23
8.25
8.26
8.29
9.0
9.28
len:22


['B_3', '8_9']
missing date
len:0


['B_3', '9_10']
missing date
7.21
len:1


['B_3', '17_18']
missing date
8.8
len:1


['B_3', '18_19']
missing date
9.4
len:1


['C_1', '8_9']
missing date
8.18
8.24
len:2


['C_1', '9_10']
missing date
7.29
8.16
8.28
8.29
len

In [95]:
names[str(i)+'_'+str('8_9')+'_'+'hr_avg']

[[132.05, 'C_3', 236, '7-19', 8, 1],
 [219.14999999999998, 'C_3', 237, '7-20', 8, 2],
 [166.15, 'C_3', 238, '7-21', 8, 3],
 [161.78666666666666, 'C_3', 240, '7-23', 8, 5],
 [207.95, 'C_3', 241, '7-24', 8, 6],
 [266.68, 'C_3', 242, '7-25', 8, 0],
 [197.34666666666666, 'C_3', 243, '7-26', 8, 1],
 [223.8, 'C_3', 244, '7-27', 8, 2],
 [294.46000000000004, 'C_3', 245, '7-28', 8, 3],
 [129.59, 'C_3', 246, '7-29', 8, 4],
 [164.52, 'C_3', 247, '7-30', 8, 5],
 [197.7, 'C_3', 248, '7-31', 8, 6],
 [227.95499999999998, 'C_3', 249, '8-1', 8, 0],
 [300.6166666666667, 'C_3', 250, '8-2', 8, 1],
 [253.25, 'C_3', 252, '8-4', 8, 3],
 [352.295, 'C_3', 253, '8-5', 8, 4],
 [209.095, 'C_3', 255, '8-7', 8, 6],
 [211.78666666666666, 'C_3', 256, '8-8', 8, 0],
 [177.24666666666667, 'C_3', 257, '8-9', 8, 1],
 [217.82, 'C_3', 258, '8-10', 8, 2],
 [218.47500000000002, 'C_3', 261, '8-13', 8, 5],
 [199.305, 'C_3', 262, '8-14', 8, 6],
 [315.09, 'C_3', 263, '8-15', 8, 0],
 [371.96, 'C_3', 265, '8-17', 8, 2],
 [374.7, 'C

In [96]:
link_route1={}
link_route1['A_2']=['110','123','107','108','120','117']
link_route1['A_3']=['110','123','107','108','119','114','118','122']
link_route1['B_1']=['105','100','111','103','116','101','121','106','113']
link_route1['B_3']=['105','100','111','103','122']
link_route1['C_1']=['115','102','109','104','112','111','103','116','101','121','106','113']
link_route1['C_3']=['115','102','109','104','112','111','103','122']

In [97]:
#把前一个小时各个link的average travel time 加入list里，如果有空值，用'null'代替（早上）
route_dict=['A_2','A_3','B_1','B_3','C_1','C_3']
for i in route_dict:
    temp_link=link_route1[i]
    for j in ['8_9','9_10']:
        names[str(i)+'_'+str(j)+'_'+'feature']=[]
        temp=names[str(i)+'_'+str(j)+'_'+'hr_avg']
        
        for k in temp:
            temp_row=k
            temp_date=k[2]
            
            for l in temp_link:
                if temp_date in link_time[l]:
                    if 7 in link_time[l][temp_date]:
                        temp2=np.mean(link_time[l][temp_date][7])
                    else: temp2='null'
                if temp_date not in link_time[l]: temp2='null'
            
                temp_row.append(temp2)
            names[str(i)+'_'+str(j)+'_'+'feature'].append(temp_row)

In [98]:
#把前一个小时各个link的average travel time 加入list里，如果有空值，用'null'代替（下午）
route_dict=['A_2','A_3','B_1','B_3','C_1','C_3']
for i in route_dict:
    temp_link=link_route1[i]
    for j in ['17_18','18_19']:
        names[str(i)+'_'+str(j)+'_'+'feature']=[]
        temp=names[str(i)+'_'+str(j)+'_'+'hr_avg']
        
        for k in temp:
            temp_row=k
            temp_date=k[2]
            
            for l in temp_link:
                if temp_date in link_time[l]:
                    if 16 in link_time[l][temp_date]:
                        temp2=np.mean(link_time[l][temp_date][16])
                    else: temp2='null'
                if temp_date not in link_time[l]: temp2='null'
            
                temp_row.append(temp2)
            names[str(i)+'_'+str(j)+'_'+'feature'].append(temp_row)
        

In [99]:
#把同一条路四个小时的信息合并
route_dict=['A_2','A_3','B_1','B_3','C_1','C_3']
time_dict=['8_9','9_10','17_18','18_19']

for i in route_dict:
    names[str(i)+'_'+'whole']=[]
    for j in time_dict:
        temp=names[str(i)+'_'+str(j)+'_'+'feature']
        for k in temp:
            names[str(i)+'_'+'whole'].append(k)
        

In [100]:
#convert to dataframe
import pandas as pd
for i in route_dict:
    temp_link=link_route1[i]
    feature=['label','route','calculated_day','date','hr','weekday']
    for j in temp_link:feature.append(j)
    names[str(i)+'_df']=pd.DataFrame(names[str(i)+'_'+'whole'], columns =feature) 

In [101]:
#找null value

for i in route_dict:
    print(i)
    result = names[str(i)+'_'+'df'].isin(['null'])

    seriesObj = result.any()
    columnNames = list(seriesObj[seriesObj == True].index)

    print('Names of columns which contains null:', columnNames)
    
    names[str(i)+'null_element']=[]
    names[str(i)+'null_row']=[]
    # Iterate over each column and fetch the rows number where
    for col in columnNames:
        rows = list(result[col][result[col] == True].index)
        for row in rows:
            names[str(i)+'null_row'].append(row)
            names[str(i)+'null_element'].append([row,col])
    print('index of rows which contains null:',set(names[str(i)+'null_row']))
    print('elements that are null:')
    print(sorted(names[str(i)+'null_element'], key = lambda x: x[0]))
   
    names[str(i)+'null_row']=list(dict.fromkeys(names[str(i)+'null_row']))
    
    print('#of rows that include null:',len(names[str(i)+'null_row']))
    print('\n')

A_2
Names of columns which contains null: []
index of rows which contains null: set()
elements that are null:
[]
#of rows that include null: 0


A_3
Names of columns which contains null: ['119', '114', '118']
index of rows which contains null: {200, 288}
elements that are null:
[[200, '119'], [200, '114'], [200, '118'], [288, '119'], [288, '114'], [288, '118']]
#of rows that include null: 2


B_1
Names of columns which contains null: []
index of rows which contains null: set()
elements that are null:
[]
#of rows that include null: 0


B_3
Names of columns which contains null: []
index of rows which contains null: set()
elements that are null:
[]
#of rows that include null: 0


C_1
Names of columns which contains null: ['115', '102', '109', '104', '112', '116', '101', '121', '106', '113']
index of rows which contains null: {33, 34, 35, 40, 41, 200, 267, 114, 274, 118, 119, 121, 124, 125}
elements that are null:
[[33, '115'], [33, '102'], [33, '109'], [33, '104'], [33, '112'], [34, '115'

In [102]:
#remove null value
for i in route_dict:
    names[str(i)+'_df']=names[str(i)+'_df'].drop(names[str(i)+'null_row'])


In [103]:
#add weather 
file_path= '/Users/vayne/Desktop/dm_pro_engin/phase1_training/weather.csv'
fr = open(file_path, 'r')
fr.readline()  # skip the header
vol_data = fr.readlines()

for i in range(len(vol_data)):
    each_pass = vol_data[i].replace('"', '').split(',')
    precipitation = float(each_pass[8])
    hour=int(each_pass[1])
    time_window = each_pass[0]
    time_window = datetime.strptime(time_window, "%Y-%m-%d")
    time_window_minute = int(math.floor(time_window.minute / 20) * 20)
    time = datetime(time_window.year, time_window.month, time_window.day, time_window.hour, time_window_minute, 0)
    if time.month*31+time.day not in weathers:
        weathers[time.month*31+time.day] = {}
    for h in range(3):
        weathers[time.month*31+time.day][hour+h]=precipitation
        
#2016.10.10 data missing let it be 0 anytime
weathers[320]={}
for i in range(24):
    weathers[320][i]=0

In [104]:
for i in route_dict:
    temp1=list(names[str(i)+'_df']['calculated_day'])
    temp2=list(names[str(i)+'_df']['hr'])
    weather_list=[]
    for j in range(len(temp1)):
        weather_list.append(weathers[temp1[j]][temp2[j]])
    names[str(i)+'_df']['precipitation']= weather_list

In [105]:
#check the data in the dataframe
for i in route_dict:
    print(i)
    print('length:'+str(len(names[str(i)+'_df'])))
    print('\n')
    print('weekday distribution')
    temp=list(names[str(i)+'_df']['weekday'])
    for j in range(0,7):
        print([temp.count(j),temp.count(j)/len(temp)])
    print('\n')
    print('hour distribution')
    for j in [8,9,17,18]:
        temp=list(names[str(i)+'_df']['hr'])
        print([temp.count(j),temp.count(j)/len(temp)])
    print('\n')

A_2
length:351


weekday distribution
[51, 0.1452991452991453]
[52, 0.14814814814814814]
[52, 0.14814814814814814]
[48, 0.13675213675213677]
[48, 0.13675213675213677]
[48, 0.13675213675213677]
[52, 0.14814814814814814]


hour distribution
[88, 0.25071225071225073]
[88, 0.25071225071225073]
[87, 0.24786324786324787]
[88, 0.25071225071225073]


A_3
length:347


weekday distribution
[51, 0.14697406340057637]
[52, 0.14985590778097982]
[52, 0.14985590778097982]
[47, 0.13544668587896252]
[48, 0.138328530259366]
[48, 0.138328530259366]
[49, 0.14121037463976946]


hour distribution
[88, 0.25360230547550433]
[87, 0.2507204610951009]
[86, 0.2478386167146974]
[86, 0.2478386167146974]


B_1
length:293


weekday distribution
[43, 0.14675767918088736]
[41, 0.13993174061433447]
[42, 0.14334470989761092]
[43, 0.14675767918088736]
[38, 0.1296928327645051]
[39, 0.13310580204778158]
[47, 0.16040955631399317]


hour distribution
[77, 0.2627986348122867]
[81, 0.2764505119453925]
[69, 0.2354948805460751]
[6

In [106]:
#output the dataframe
for i in route_dict:
    names[str(i)+'_df'].to_csv(str(i)+'.csv')


In [107]:
names[str(i)+'_df']

,label,route,calculated_day,date,hr,weekday,115,102,109,104,112,111,103,122,precipitation
0,132.050000,C_3,236,7-19,8,1,6.70333,9.97667,10.7433,25.3133,30.3433,52.822500,2.505833,30.864286,0.0
1,219.150000,C_3,237,7-20,8,2,17.8333,12.845,13.2183,32.235,23.7033,36.149474,11.691053,63.107000,0.0
2,166.150000,C_3,238,7-21,8,3,9.85333,11.21,11.4667,25.16,15.0667,31.747143,12.790000,42.614762,0.0
3,161.786667,C_3,240,7-23,8,5,16.76,11.8329,11.8286,26.5014,19.0214,29.680000,10.785909,43.795000,0.0
4,207.950000,C_3,241,7-24,8,6,10.0983,11.8333,12.0733,23.0283,17.815,32.623000,8.675000,31.590909,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,178.660000,C_3,323,10-13,18,3,9.96429,10.2238,9.64714,25.4786,25.1357,26.789070,5.398372,37.200741,0.0
288,152.910000,C_3,324,10-14,18,4,10.0636,10.3032,10.8645,30.3,26.651,26.421639,5.395574,34.268615,0.0
289,250.595000,C_3,325,10-15,18,5,15.5961,11.1643,9.33652,32.8626,28.4365,37.797869,13.875500,38.566250,0.0
290,142.770000,C_3,326,10-16,18,6,13.1061,11.2889,18.3594,51.9411,25.9944,25.178627,5.083725,36.584833,0.0
